In [119]:
import pandas as pd

In [120]:
df = pd.read_csv('../notion_data/코스피지수.csv')

In [121]:
df.head()

,변환,원자료
0,2014-12-30,"1,915.59"
1,2015-01-02,"1,926.44"
2,2015-01-05,"1,915.75"
3,2015-01-06,"1,882.45"
4,2015-01-07,"1,883.83"


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   변환      2469 non-null   object
 1   원자료     2459 non-null   object
dtypes: object(2)
memory usage: 38.7+ KB


In [123]:
# 결측치 있는 행 추출

df[df.isna().any(axis=1)]

,변환,원자료
249,2015-12-31,NaN
496,2016-12-30,NaN
740,2017-12-29,NaN
985,2018-12-31,NaN
1232,2019-12-31,NaN
1481,2020-12-31,NaN
1730,2021-12-31,NaN
1977,2022-12-30,NaN
2223,2023-12-29,NaN
2468,2024-12-31,NaN


In [124]:
df2 = df.copy()

In [125]:
df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2459 entries, 0 to 2467
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   변환      2459 non-null   object
 1   원자료     2459 non-null   object
dtypes: object(2)
memory usage: 57.6+ KB


In [126]:
df.tail()

,변환,원자료
2463,2024-12-23,"2,442.01"
2464,2024-12-24,"2,440.52"
2465,2024-12-26,"2,429.67"
2466,2024-12-27,"2,404.77"
2467,2024-12-30,"2,399.49"


In [127]:
import pandas as pd

# 문자열 숫자를 float으로 변환
df['원자료'] = df['원자료'].str.replace(',', '').astype(float)

# 날짜 변환 및 정렬
df['변환'] = pd.to_datetime(df['변환'])
df.sort_values('변환', inplace=True)
df.set_index('변환', inplace=True)

# 일별 수익률 계산: 12월 30일은 제외하고 그 다음부터 계산
df['일별 수익률'] = df['원자료'].pct_change()
df.loc[df.index[0], '일별 수익률'] = None  # 첫날은 NaN

# 월말 종가 기준으로 월별 수익률 계산
monthly_close = df['원자료'].resample('M').last()
monthly_return = monthly_close.pct_change()

# 월별 수익률을 원본에 병합
df['월'] = df.index.to_period('M')
df = df.merge(monthly_return.rename('월별 수익률'), left_on='월', right_index=True, how='left')

# 불필요한 컬럼 제거 (선택)
df.drop(columns=['월'], inplace=True)

# 출력
print(df.head(10))


                원자료    일별 수익률  월별 수익률
변환                                   
2014-12-30  1915.59       NaN     NaN
2015-01-02  1926.44  0.005664     NaN
2015-01-05  1915.75 -0.005549     NaN
2015-01-06  1882.45 -0.017382     NaN
2015-01-07  1883.83  0.000733     NaN
2015-01-08  1904.65  0.011052     NaN
2015-01-09  1924.70  0.010527     NaN
2015-01-12  1920.95 -0.001948     NaN
2015-01-13  1917.14 -0.001983     NaN
2015-01-14  1913.66 -0.001815     NaN


C:\Users\Ph.D.Huh\AppData\Local\Temp\ipykernel_7520\3844182659.py:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_close = df['원자료'].resample('M').last()


In [128]:
print(df.head(10))

                원자료    일별 수익률  월별 수익률
변환                                   
2014-12-30  1915.59       NaN     NaN
2015-01-02  1926.44  0.005664     NaN
2015-01-05  1915.75 -0.005549     NaN
2015-01-06  1882.45 -0.017382     NaN
2015-01-07  1883.83  0.000733     NaN
2015-01-08  1904.65  0.011052     NaN
2015-01-09  1924.70  0.010527     NaN
2015-01-12  1920.95 -0.001948     NaN
2015-01-13  1917.14 -0.001983     NaN
2015-01-14  1913.66 -0.001815     NaN


In [129]:
df = df.drop(columns=['월별 수익률'])

In [130]:
df.head(10)

,원자료,일별 수익률
변환,,
2014-12-30,1915.59,NaN
2015-01-02,1926.44,0.005664
2015-01-05,1915.75,-0.005549
2015-01-06,1882.45,-0.017382
2015-01-07,1883.83,0.000733
2015-01-08,1904.65,0.011052
2015-01-09,1924.70,0.010527
2015-01-12,1920.95,-0.001948
2015-01-13,1917.14,-0.001983


In [131]:
import pandas as pd

# 날짜 정렬
df = df.sort_index()

# 월별로 일자순번 붙이기 (월별로 1부터 순번)
df['일자순번'] = df.groupby(df.index.to_period('M')).cumcount() + 1

# 기간구간 함수 정의
def 기간구간(day):
    if 1 <= day <= 5:
        return '1~5일'
    elif 6 <= day <= 10:
        return '6~10일'
    elif 11 <= day <= 15:
        return '11~15일'
    elif 16 <= day <= 20:
        return '16~20일'
    else:
        return '기타'

df['기간구간'] = df['일자순번'].apply(기간구간)

# 년월 컬럼 추가
df['년월'] = df.index.to_period('M')

# 월별, 기간구간별 표준편차 계산 및 추가
df['수익률_표준편차'] = df.groupby(['년월', '기간구간'])['일별 수익률'].transform('std')

result = df[['원자료', '일별 수익률', '일자순번', '기간구간', '년월', '수익률_표준편차']]
print(result)


                원자료    일별 수익률  일자순번    기간구간       년월  수익률_표준편차
변환                                                            
2014-12-30  1915.59       NaN     1    1~5일  2014-12       NaN
2015-01-02  1926.44  0.005664     1    1~5일  2015-01  0.010974
2015-01-05  1915.75 -0.005549     2    1~5일  2015-01  0.010974
2015-01-06  1882.45 -0.017382     3    1~5일  2015-01  0.010974
2015-01-07  1883.83  0.000733     4    1~5일  2015-01  0.010974
...             ...       ...   ...     ...      ...       ...
2024-12-23  2442.01  0.015748    16  16~20일  2024-12  0.009713
2024-12-24  2440.52 -0.000610    17  16~20일  2024-12  0.009713
2024-12-26  2429.67 -0.004446    18  16~20일  2024-12  0.009713
2024-12-27  2404.77 -0.010248    19  16~20일  2024-12  0.009713
2024-12-30  2399.49 -0.002196    20  16~20일  2024-12  0.009713

[2459 rows x 6 columns]


In [132]:
df.head(30)

,원자료,일별 수익률,일자순번,기간구간,년월,수익률_표준편차
변환,,,,,,
2014-12-30,1915.59,NaN,1,1~5일,2014-12,NaN
2015-01-02,1926.44,0.005664,1,1~5일,2015-01,0.010974
2015-01-05,1915.75,-0.005549,2,1~5일,2015-01,0.010974
2015-01-06,1882.45,-0.017382,3,1~5일,2015-01,0.010974
2015-01-07,1883.83,0.000733,4,1~5일,2015-01,0.010974
2015-01-08,1904.65,0.011052,5,1~5일,2015-01,0.010974
2015-01-09,1924.70,0.010527,6,6~10일,2015-01,0.005405
2015-01-12,1920.95,-0.001948,7,6~10일,2015-01,0.005405
2015-01-13,1917.14,-0.001983,8,6~10일,2015-01,0.005405


In [133]:
result_df = df[['원자료', '일별 수익률', '기간구간', '수익률_표준편차']]
result_df.head(30)

,원자료,일별 수익률,기간구간,수익률_표준편차
변환,,,,
2014-12-30,1915.59,NaN,1~5일,NaN
2015-01-02,1926.44,0.005664,1~5일,0.010974
2015-01-05,1915.75,-0.005549,1~5일,0.010974
2015-01-06,1882.45,-0.017382,1~5일,0.010974
2015-01-07,1883.83,0.000733,1~5일,0.010974
2015-01-08,1904.65,0.011052,1~5일,0.010974
2015-01-09,1924.70,0.010527,6~10일,0.005405
2015-01-12,1920.95,-0.001948,6~10일,0.005405
2015-01-13,1917.14,-0.001983,6~10일,0.005405


In [134]:
result_df = result_df[result_df['기간구간'] != '기타']


In [135]:
result_df.head(30)

,원자료,일별 수익률,기간구간,수익률_표준편차
변환,,,,
2014-12-30,1915.59,NaN,1~5일,NaN
2015-01-02,1926.44,0.005664,1~5일,0.010974
2015-01-05,1915.75,-0.005549,1~5일,0.010974
2015-01-06,1882.45,-0.017382,1~5일,0.010974
2015-01-07,1883.83,0.000733,1~5일,0.010974
2015-01-08,1904.65,0.011052,1~5일,0.010974
2015-01-09,1924.70,0.010527,6~10일,0.005405
2015-01-12,1920.95,-0.001948,6~10일,0.005405
2015-01-13,1917.14,-0.001983,6~10일,0.005405


In [ ]:
# result_df.to_csv('코스피_수익률_표준편차.csv', encoding='cp949')

In [141]:
pre_df = pd.read_csv('../data/pre_data.csv', encoding='utf-8')

In [142]:
pre_df.to_csv('pre_df.csv', encoding='utf-8')

In [143]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('../notion_data/코스피지수.csv')
df['원자료'] = df['원자료'].str.replace(',', '').astype(float)
df['변환'] = pd.to_datetime(df['변환'])
df.sort_values('변환', inplace=True)
df.set_index('변환', inplace=True)

# 수익률 계산
df['일별 수익률'] = df['원자료'].pct_change()
df.loc[df.index[0], '일별 수익률'] = None

# 일자순번 및 기간 구간 구하기
df = df.sort_index()
df['일자순번'] = df.groupby(df.index.to_period('M')).cumcount() + 1

def 기간구간(day):
    if 1 <= day <= 5:
        return '1~5일'
    elif 6 <= day <= 10:
        return '6~10일'
    elif 11 <= day <= 15:
        return '11~15일'
    elif 16 <= day <= 20:
        return '16~20일'
    else:
        return '기타'

df['기간구간'] = df['일자순번'].apply(기간구간)
df['년월'] = df.index.to_period('M')

# 기간구간별 표준편차 계산
df['수익률_표준편차'] = df.groupby(['년월', '기간구간'])['일별 수익률'].transform('std')

# 피벗 테이블로 변환
col_name_map = {
    '1~5일': '원자료(-1~-5)',
    '6~10일': '원자료(-6~-10)',
    '11~15일': '원자료(-11~-15)',
    '16~20일': '원자료(-16~-20)',
}
pivot_std = df[['수익률_표준편차', '기간구간']].pivot(columns='기간구간', values='수익률_표준편차')
pivot_std.rename(columns=col_name_map, inplace=True)

# 결과 보기
pivot_std.head()


C:\Users\Ph.D.Huh\AppData\Local\Temp\ipykernel_7520\3968124156.py:11: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['일별 수익률'] = df['원자료'].pct_change()


기간구간,원자료(-11~-15),원자료(-16~-20),원자료(-1~-5),원자료(-6~-10),기타
변환,,,,,
2014-12-30,NaN,NaN,NaN,NaN,NaN
2015-01-02,NaN,NaN,0.010974,NaN,NaN
2015-01-05,NaN,NaN,0.010974,NaN,NaN
2015-01-06,NaN,NaN,0.010974,NaN,NaN
2015-01-07,NaN,NaN,0.010974,NaN,NaN


In [144]:
pivot_std.head(30)

기간구간,원자료(-11~-15),원자료(-16~-20),원자료(-1~-5),원자료(-6~-10),기타
변환,,,,,
2014-12-30,NaN,NaN,NaN,NaN,NaN
2015-01-02,NaN,NaN,0.010974,NaN,NaN
2015-01-05,NaN,NaN,0.010974,NaN,NaN
2015-01-06,NaN,NaN,0.010974,NaN,NaN
2015-01-07,NaN,NaN,0.010974,NaN,NaN
2015-01-08,NaN,NaN,0.010974,NaN,NaN
2015-01-09,NaN,NaN,NaN,0.005405,NaN
2015-01-12,NaN,NaN,NaN,0.005405,NaN
2015-01-13,NaN,NaN,NaN,0.005405,NaN


In [ ]:
df = pivot_std[['원자료(-1~-5)', '원자료(-6~-10)', '원자료(-11~-15)', '원자료(-16~-20)', '기타']]
df.head(30)

In [151]:
df = df.rename(columns={
    '원자료(-1~-5)' : '코스피(-1~-5)',
    '원자료(-6~-10)' : '코스피(-6~-10)',
    '원자료(-11~-15)' : '코스피(-11~-15)',
    '원자료(-16~-20)' : '코스피(-16~-20)',
    '기타' : '기타'
})


In [ ]:
df.head()

기간구간,코스피(-1~-5),코스피(-6~-10),코스피(-11~-15),코스피(-16~-20),기타
변환,,,,,
2014-12-30,NaN,NaN,NaN,NaN,NaN
2015-01-02,0.010974,NaN,NaN,NaN,NaN
2015-01-05,0.010974,NaN,NaN,NaN,NaN
2015-01-06,0.010974,NaN,NaN,NaN,NaN
2015-01-07,0.010974,NaN,NaN,NaN,NaN
